Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

/home/fja0568/src/brownbear
/home/fja0568/src/brownbear/universe/sp400-galaxy


In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Ticker symbol': 'Symbol',
                      'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
AAN,Aaron's Inc.,Consumer Discretionary,Homefurnishing Retail
ACHC,Acadia Healthcare,Health Care,Health Care Facilities
ACIW,ACI Worldwide,Information Technology,Application Software
ADNT,Adient plc,Consumer Discretionary,Auto Parts & Equipment
ATGE,Adtalem Global Education,Consumer Discretionary,Education Services
...,...,...,...
WYND,Wyndham Destinations,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
X,United States Steel Corporation,Materials,Steel
Y,Alleghany Corporation,Financials,Reinsurance


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# map sp400 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
sp400.index = sp400.index.str.replace('.', '-')
sp400

,Description,Asset Class,GICS Sub Industry
Symbol,,,
AAN,Aaron's Inc.,US Stocks:Consumer Discretionary,Homefurnishing Retail
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities
ACIW,ACI Worldwide,US Stocks:Technology,Application Software
ADNT,Adient plc,US Stocks:Consumer Discretionary,Auto Parts & Equipment
ATGE,Adtalem Global Education,US Stocks:Consumer Discretionary,Education Services
...,...,...,...
WYND,Wyndham Destinations,US Stocks:Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
X,United States Steel Corporation,US Stocks:Materials,Steel
Y,Alleghany Corporation,US Stocks:Financials,Reinsurance


In [8]:
# drop invalid symbols
sp400.drop(['SNH', 'FII', 'HPT', 'ISCA', 'VSM', 'AKRX', 'APY', 'POL'], inplace=True)

In [9]:
# make symbols list
symbols = list(sp400.index)
#symbols

In [10]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

........................................................................................................................................................................................................................................................................................................................................................................................................


In [11]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AAN,ACHC,ACIW,ADNT,ATGE,ACM,AMG,ACC,AEO,AFG,...,WTFC,WTRG,WW,WWD,WWE,WYND,X,Y,XPO,YELP
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,nan,59.98,19.71,nan,43.01,30.35,203.98,32.82,11.46,48.33,...,43.15,23.29,21.53,46.48,10.90,31.78,25.12,444.64,40.63,55.15
2015-01-05,nan,59.12,19.26,nan,43.60,29.02,196.58,33.01,11.59,47.70,...,42.31,22.84,21.26,45.48,10.70,31.21,23.95,435.03,39.37,52.53
2015-01-06,nan,58.19,18.95,nan,41.75,28.73,192.35,33.36,11.63,47.43,...,41.26,22.86,19.74,45.15,10.73,30.83,23.22,432.47,37.98,52.44
2015-01-07,nan,60.63,19.03,nan,42.04,29.32,197.07,33.41,12.20,47.74,...,41.62,23.07,20.35,45.57,10.28,31.41,23.28,434.10,38.14,52.21
2015-01-08,nan,61.76,19.01,nan,42.91,30.23,199.07,33.61,11.63,48.61,...,41.95,23.22,20.32,45.34,10.17,32.23,23.79,437.79,38.35,53.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-30,19.19,42.45,32.58,31.29,28.63,51.89,87.12,39.80,17.99,89.41,...,54.49,45.28,29.51,111.83,43.04,42.06,14.19,575.20,106.68,31.94
2020-12-01,18.41,41.98,33.42,33.12,28.64,52.46,88.74,40.31,17.90,90.18,...,55.89,46.72,29.93,113.51,44.86,43.96,15.06,590.75,110.42,32.11
2020-12-02,18.25,42.28,37.04,32.73,28.61,52.54,90.06,40.49,18.35,90.15,...,57.65,47.37,31.26,115.34,44.48,44.84,15.57,593.34,110.01,31.51


In [12]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['AAN']

nan

In [13]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [14]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['AAN']

1.1513458331238715

In [15]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['AAN']

1.048896315971717

In [16]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['AAN'])

Date
2015-01-31     nan
2015-02-28     nan
2015-03-31     nan
2015-04-30     nan
2015-05-31     nan
2015-06-30     nan
2015-07-31     nan
2015-08-31     nan
2015-09-30     nan
2015-10-31     nan
2015-11-30     nan
2015-12-31     nan
2016-01-31     nan
2016-02-29     nan
2016-03-31     nan
2016-04-30     nan
2016-05-31     nan
2016-06-30     nan
2016-07-31     nan
2016-08-31     nan
2016-09-30     nan
2016-10-31     nan
2016-11-30     nan
2016-12-31     nan
2017-01-31     nan
2017-02-28     nan
2017-03-31     nan
2017-04-30     nan
2017-05-31     nan
2017-06-30     nan
2017-07-31     nan
2017-08-31     nan
2017-09-30     nan
2017-10-31     nan
2017-11-30     nan
2017-12-31     nan
2018-01-31     nan
2018-02-28     nan
2018-03-31     nan
2018-04-30     nan
2018-05-31     nan
2018-06-30     nan
2018-07-31     nan
2018-08-31     nan
2018-09-30     nan
2018-10-31     nan
2018-11-30     nan
2018-12-31     nan
2019-01-31     nan
2019-02-28     nan
2019-03-31     nan
2019-04-30     nan
2019-05

In [17]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['AAN']

Date
2015-01-31     nan
2015-02-28     nan
2015-03-31     nan
2015-04-30     nan
2015-05-31     nan
              ... 
2020-08-31     nan
2020-09-30     nan
2020-10-31     nan
2020-11-30     nan
2020-12-31   -0.01
Freq: M, Name: AAN, Length: 72, dtype: float64

In [18]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['AAN']

0.0

In [19]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 400 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

# This is still slow (2.53 s)
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')